In [21]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [22]:
# read in the IJH_2010.parquet.textClipping file
df = pd.read_parquet('IJH_2010.parquet')
df.head()

prcld   eps  prcod     cshtrd      cusip  gsector  \
date       id                                                             
2010-06-30 064356_01  12.60 -2.10  13.01  4182555.0  171779309     45.0   
           063080_01  40.49  1.44  41.21   379908.0  03662Q105     45.0   
           118267_01  13.83  0.12  14.09   276324.0  500643200     20.0   
           010540_01  34.60  2.11  35.33   436688.0  884315102     20.0   
           024409_01  64.50  2.15  64.51  2398964.0  871130100     45.0   

                      prccd        cik   tic  prchd       cshoc  
date       id                                                    
2010-06-30 064356_01  12.68   936395.0  CIEN  13.18  93094000.0  
           063080_01  40.57  1013462.0  ANSS  41.57  90509000.0  
           118267_01  13.90    56679.0   KFY  14.30  45449000.0  
           010540_01  34.70    97854.0   TNB  35.69  52795000.0  
           024409_01  64.66   768262.0  SY.3  64.73  86861000.0

In [23]:
new_index = []
# 1. Iterate over the index values of the dataframe
# 2. For each index value, get the row of the dataframe
# 3. Get the value of the 'tic' column of the row
# 4. Append the tuple (index value, tic value) to the new_index list
for tup in df.index.values:
    new_index.append((tup[0], df.loc[tup]['tic']))

In [25]:
# Create a new MultiIndex using the new_index list of tuples
new_idx = pd.MultiIndex.from_tuples(new_index, names=['date','ticker'])

In [26]:
# Making a new dataframe where the column are the ticker names and the rows are the close prices for each day.
df.set_index(new_idx, inplace=True)
#df.drop(columns=['id'], axis=1, inplace=True)
df.index
df.head()

prcld   eps  prcod     cshtrd      cusip  gsector  prccd  \
date       ticker                                                             
2010-06-30 CIEN    12.60 -2.10  13.01  4182555.0  171779309     45.0  12.68   
           ANSS    40.49  1.44  41.21   379908.0  03662Q105     45.0  40.57   
           KFY     13.83  0.12  14.09   276324.0  500643200     20.0  13.90   
           TNB     34.60  2.11  35.33   436688.0  884315102     20.0  34.70   
           SY.3    64.50  2.15  64.51  2398964.0  871130100     45.0  64.66   

                         cik   tic  prchd       cshoc  
date       ticker                                      
2010-06-30 CIEN     936395.0  CIEN  13.18  93094000.0  
           ANSS    1013462.0  ANSS  41.57  90509000.0  
           KFY       56679.0   KFY  14.30  45449000.0  
           TNB       97854.0   TNB  35.69  52795000.0  
           SY.3     768262.0  SY.3  64.73  86861000.0

In [27]:
df = df.drop(columns=['tic'], axis=1)
df.drop(columns=['prcld', 'eps','prccd', 'cshtrd', 'cusip', 'gsector', 'cik', 'prchd', 'cshoc'], axis=1, inplace=True)
df.head()

prcod
date       ticker       
2010-06-30 CIEN    13.01
           ANSS    41.21
           KFY     14.09
           TNB     35.33
           SY.3    64.51

In [28]:
unstacked_close = df.stack().unstack(level=1) # stack and unstack at second level of multiindex as an inner transpose
date_index = list(OrderedDict.fromkeys([tup[0] for tup in df.index.values])) # convert multiindex to single index

Beginning of backtest

In [29]:
unstacked_close.set_index(pd.Index(date_index, name='date').astype(int), inplace=True) # set new index as integer date
unstacked_close.head()

ticker,AA,AAI,AAN.2,AAP,ABMD,ACC,ACF,ACHC,ACIIQ,ACIW,...,X,XEC,XLS,XPO,XRX,Y,YELP,YETI,ZBRA,ZD
date,,,,,,,,,,,,,,,,,,,,,
1277856000000000000,NaN,4.96,17.28,50.29,NaN,NaN,18.50,NaN,198.2,6.470000,...,NaN,71.70,NaN,NaN,NaN,NaN,NaN,NaN,26.03,NaN
1277942400000000000,NaN,4.85,17.00,50.38,NaN,NaN,18.87,NaN,197.5,6.473333,...,NaN,71.64,NaN,NaN,NaN,NaN,NaN,NaN,25.36,NaN
1278028800000000000,NaN,4.83,16.85,51.72,NaN,NaN,18.65,NaN,198.5,6.720000,...,NaN,70.55,NaN,NaN,NaN,NaN,NaN,NaN,24.80,NaN
1278374400000000000,NaN,4.87,16.98,51.80,NaN,NaN,18.91,NaN,197.6,6.540000,...,NaN,71.58,NaN,NaN,NaN,NaN,NaN,NaN,24.86,NaN
1278460800000000000,NaN,4.64,16.42,50.67,NaN,NaN,18.40,NaN,198.5,6.406667,...,NaN,71.38,NaN,NaN,NaN,NaN,NaN,NaN,24.34,NaN


In [30]:
df = unstacked_close

# Beginning of backtest

In [31]:
# calculate the daily returns
daily_returns = df.pct_change()  # calculate the daily returns
mkt_daily_returns = daily_returns.mean(axis=1)  # calculate the market return, which is also mean of the daily returns

In [32]:
# calculate the weights
weights = -(np.array(daily_returns) - np.array(mkt_daily_returns).reshape((daily_returns.shape[0], 1)))  # calculate the weights
weights_sum = np.nansum(abs(weights), axis=1)  # calculate the sum of the weights
weights[weights_sum == 0,] = 0  # set the weights to zero if the sum of the weights is zero
weights_sum[weights_sum == 0] = 1  # set the sum of the weights to one if the sum of the weights is zero
weights = weights / weights_sum.reshape((daily_returns.shape[0], 1))  # normalize the weights

In [33]:
startDate = 1277856000000000000
endDate = 1667260800000000000
# Get the daily pnl
daily_pnl = np.nansum(np.array(pd.DataFrame(weights).shift())* np.array(daily_returns), axis=1)  # calculate the daily pnl
daily_pnl = daily_pnl[np.logical_and(df.index >= startDate, df.index <= endDate)]  # filter the daily pnl by the start and end date

In [34]:
# Calculate the sharpe ratio
sharpe_ratio = np.sqrt(252) * np.mean(daily_pnl) / np.std(daily_pnl)  # calculate the sharpe ratio
print(f'Sharpe without transaction costs: {sharpe_ratio}')

Sharpe without transaction costs: 0.5099866064134727


In [35]:
# Calculate the sharpe ratio with transaction costs
transaction_costs = 0.0005  # set the transaction costs
weights = weights[np.logical_and(df.index >= startDate, df.index <= endDate)]  # filter the weights by the start and end date
daily_pnl_with_costs = daily_pnl - (np.nansum(abs(weights - np.array(pd.DataFrame(weights).shift())), axis=1) * transaction_costs)  # calculate the daily pnl with transaction costs
sharpe_ratio_with_costs = np.sqrt(252) * np.mean(daily_pnl_with_costs) / np.std(daily_pnl_with_costs)  # calculate the sharpe ratio with transaction costs
print(f'Sharpe with transaction costs: {sharpe_ratio_with_costs}')

Sharpe with transaction costs: -1.3840453047282346


Summary (closes prices):
- sharpe w/out transaction costs: 0.2489251722768505
- sharpe w transaction costs: -1.5764436739757899

# Running the same steps with the open prices achieves:

Summary (open prices):
- sharpe w/out transaction costs: 0.5099866064134727
- sharpe w transaction costs: -1.3840453047282346